# Tools
This notebook contains some usefulclasses and functions coded so far

In [1]:
import numpy as np

In [4]:
class Dense(object):
    """A class representing a Dense layer in the neural network"""
    @staticmethod
    def relu(x):
        return max(0, x)
    
    @staticmethod
    def softmax(inputs):
        """Computes probabilities given the VALUES"""
        exp_values = np.exp(inputs - np.max(inputs, axis=1,
        keepdims=True))
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1,
        keepdims=True)
        return probabilities
    
    @staticmethod
    def _categorical_cross_entropy(predicted, correct):
        # Number of samples in a batch
        samples = len(predicted)
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        predicted_clipped = np.clip(predicted, 1e-7, 1 - 1e-7)
        # Probabilities for target values -
        # only if categorical labels
        if len(correct.shape) == 1:
            correct_confidences = predicted_clipped[
            range(samples),
            correct
            ]
            # Mask values - only for one-hot encoded labels
        elif len(correct.shape) == 2:
            correct_confidences = np.sum(
            predicted_clipped * correct,
            axis=1
            )
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return np.mean(negative_log_likelihoods)
    
    def _accuracy(predictions, targets):
        # Calculate values along second axis (axis of index 1)
        predictions = np.argmax(predictions, axis=1)
        # If targets are one-hot encoded - convert them
        if len(class_targets.shape) == 2:
            targets = np.argmax(targets, axis=1)
        # True evaluates to 1; False to 0
        return np.mean(predictions==targets)
    
    
    def __init__(self, inputs: int, neurons: int, activation):
        # Initialize weights randomly
        # Each COLUMN in the resulting matrix is a neuron's weights
        # It is done to avoid transposing the weights matrix every time we make a forward pass
        # np.random.randn produces a Gaussian distribution with mean of 0 and variance of 1
        self.weights = 0.01 * np.random.randn(inputs, neurons)
        # Biases default to zero
        self.biases = np.zeros((1, neurons))
        # Set activation function
        if isinstance(activation, str):
            if hasattr(self, activation):
                self.activation = getattr(self, activation)
            else:
                raise ValueError(f'Invalid function "{activation}": no such function is defined! You might want to pass in the function object instead')
        elif callable(activation):
            self.activation = activation
        else:
            raise ValueError(f'"Activation" parameter must be either a string specifying activation function name or a callable')
    
    def forward(self, inputs):
        self.output = self.activation(np.dot(inputs, self.weights) + self.biases)
        return self.output
    
    @property
    def accuracy(self, targets):
        return self._accuracy(self.output, targets)
    
    @property
    def crossentropy_loss(self, targets):
        return self._categorical_cross_entropy(self.output, targets)
        